<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Purpose" data-toc-modified-id="Purpose-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Purpose</a></span></li><li><span><a href="#Notes:" data-toc-modified-id="Notes:-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Notes:</a></span></li><li><span><a href="#Input" data-toc-modified-id="Input-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Input</a></span></li><li><span><a href="#Output" data-toc-modified-id="Output-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Output</a></span></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#Cell-meta" data-toc-modified-id="Cell-meta-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Cell meta</a></span></li><li><span><a href="#MCDS" data-toc-modified-id="MCDS-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>MCDS</a></span></li></ul></li><li><span><a href="#Filter-feature" data-toc-modified-id="Filter-feature-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Filter feature</a></span><ul class="toc-item"><li><span><a href="#Remove-chromosome" data-toc-modified-id="Remove-chromosome-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Remove chromosome</a></span></li><li><span><a href="#Remove-blacklist-region" data-toc-modified-id="Remove-blacklist-region-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Remove blacklist region</a></span></li><li><span><a href="#Remove-by-mean-cov" data-toc-modified-id="Remove-by-mean-cov-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Remove by mean cov</a></span></li></ul></li><li><span><a href="#Add-mC-rate" data-toc-modified-id="Add-mC-rate-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Add mC rate</a></span></li><li><span><a href="#Select-HVF" data-toc-modified-id="Select-HVF-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Select HVF</a></span><ul class="toc-item"><li><span><a href="#mCH-HVF" data-toc-modified-id="mCH-HVF-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>mCH HVF</a></span></li><li><span><a href="#mCG-HVF" data-toc-modified-id="mCG-HVF-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>mCG HVF</a></span></li></ul></li><li><span><a href="#Get-Anndata" data-toc-modified-id="Get-Anndata-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Get Anndata</a></span><ul class="toc-item"><li><span><a href="#Load-HVF" data-toc-modified-id="Load-HVF-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Load HVF</a></span></li><li><span><a href="#CH-adata" data-toc-modified-id="CH-adata-10.2"><span class="toc-item-num">10.2&nbsp;&nbsp;</span>CH adata</a></span></li><li><span><a href="#CG-adata" data-toc-modified-id="CG-adata-10.3"><span class="toc-item-num">10.3&nbsp;&nbsp;</span>CG adata</a></span></li></ul></li><li><span><a href="#Prepare-Gene" data-toc-modified-id="Prepare-Gene-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Prepare Gene</a></span></li></ul></div>

# Prepare AnnData and Highly Variable Feature Selection

## Purpose
- Calculate methylation rate
- Normalize per cell and clip extreme
- Highly variable feature (usually 100kb bins) selection
- Prepare HVF adata file
- Prepare gene adata file
- Basically, from N-D MCDS to 2-D anndata.AnnData object

## Notes:
- Usually we use 100kb bins to do clustering and use gene body mCH (for neuron) or mCG (for non-neuron) to annotate clusters.
- Therefore, feature usually refer to chrom100k bins, but you can try to use other feature (e.g. gene) to do clustering.

## Input
- Cell metadata table, MCDS list

## Output
- mCH HVF adata
- mCG HVF adata
- Gene rate MCDS

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy.api as sc
import seaborn as sns
import xarray as xr
from pybedtools import BedTool
from ALLCools.mcds.MCDS import MCDS
from cemba_data.plot import *

In [3]:
result_dir = pathlib.Path('Adata')
result_dir.mkdir(exist_ok=True)
fig_dir = pathlib.Path('fig/feature_selection')
fig_dir.mkdir(exist_ok=True, parents=True)

## Parameters

In [4]:
# parameters cell
in_memory = True
dask_distribute = False

# selected cell metadata path
cell_meta_path = ''

# blacklist
black_list_path = '/home/hanliu/project/mouse_rostral_brain/misc/mm10-blacklist.v2.bed.gz'

# mcds_path
mcds_path_list = []

clustering_feature = 'chrom100k'  # usually 100kb chromosome bins or genes

# remove bad features
black_list_region = None
exclude_chromosome = ['chrY', 'chrM']

# preprocess parameters
min_feature_cov, max_feature_cov = 500, 3000

ch_hvf_top = 3000
min_ch_hvf_mean = 0.5
max_ch_hvf_mean = 2.5

cg_hvf_top = 3000
min_cg_hvf_mean = 0.5
max_cg_hvf_mean = 1.2

In [5]:
# Parameters
in_memory = False
dask_distribute = True
cell_meta_path = "/home/hanliu/project/mouse_rostral_brain/study/Level3-SubCellType/MGC/CellMetadata.AfterQC.msg"
mcds_path_list = ["/home/hanliu/project/mouse_rostral_brain/dataset/4H-180806.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/11E-190214.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5A-190205.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/11E-190305.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4H-180911.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/11F-190214.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5A-190307.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/11F-190305.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5B-180514.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/1A-180226.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5B-180529.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/1A-180227.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5F-181218.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/1B-180119.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5F-181220.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/1B-180213.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5G-181008.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/1C-180208.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5G-181009.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/1C-180212.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5H-181015.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/2A-180122.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5H-181016.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/2A-180123.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5J-180904.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/2B-180305.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5J-180910.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/2B-180306.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5J-190207.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/2C-180409.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/6A-190108.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/2C-180410.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/6A-190117.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/2D-180313.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/6B-181022.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/2D-180319.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/6B-181023.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/2D-180403.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/6C-190122.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/2E-180220.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/6C-190124.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/2E-180222.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/6D-190131.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/3A-180129.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/6D-190205.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/3A-180130.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/7B-180423.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/3B-180312.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/7B-180424.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/3B-180501.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/8B-180426.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/3C-171206.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/8B-180430.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/3C-171207.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/9H-190212.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/3D-180412.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/9H-190219.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/3D-180416.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/9J-190212.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/3E-180320.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/9J-190219.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/3E-180326.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/11B-190314.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/3F-180109.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/11B-190325.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/3F-180405.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/12B-190314.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4A-180205.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/12B-190325.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4A-180206.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/9B-190326.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4B-171212.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/9B-190404.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4B-171213.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/9D-190326.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4B-180104.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/9D-190404.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4C-180417.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/10E-190625.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4C-180419.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/10E-190627.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4D-171214.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/10F-190625.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4D-171219.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/10F-190627.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4E-180110.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/10G-190423.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4E-180111.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/10G-190523.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4F-180329.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/8E-190711.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4F-180402.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/8E-190716.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4G-180917.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/8J-190711.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4G-181204.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/8J-190716.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/4H-180724.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5C-181002.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5C-181210.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5D-180605.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5D-180612.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5E-180925.mcds", "/home/hanliu/project/mouse_rostral_brain/dataset/5E-181206.mcds"]
clustering_feature = "chrom100k"
black_list_region = None
exclude_chromosome = ["chrY", "chrM"]
min_feature_cov = 500
max_feature_cov = 3000
ch_hvf_top = 6000
min_ch_hvf_mean = 0.5
max_ch_hvf_mean = 2.5
cg_hvf_top = 6000
min_cg_hvf_mean = 0.5
max_cg_hvf_mean = 1.2


In [6]:
if dask_distribute:
    from dask.distributed import Client
    client = Client(dashboard_address=':5555')

## Load data
### Cell meta

In [7]:
cell_meta = pd.read_msgpack('CellMetadata.AfterQC.msg')

### MCDS

In [8]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    mcds = MCDS.open(mcds_path_list,
                     use_cells=cell_meta[cell_meta['PassFilter']].index,
                     chunks={'cell': 1000})
if in_memory:
    mcds.load()

In [9]:
mcds

<xarray.MCDS>
Dimensions:              (cell: 853, chrom100k: 27269, count_type: 2, gene: 55487, mc_type: 2)
Coordinates:
  * mc_type              (mc_type) object 'CGN' 'CHN'
  * gene                 (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
  * count_type           (count_type) object 'mc' 'cov'
    strand_type          <U4 'both'
    gene_chrom           (gene) object dask.array<shape=(55487,), chunksize=(55487,)>
    gene_start           (gene) int64 dask.array<shape=(55487,), chunksize=(55487,)>
    gene_end             (gene) int64 dask.array<shape=(55487,), chunksize=(55487,)>
  * cell                 (cell) object '10E_M_1717' '10E_M_1724' ... '9J_M_794'
  * chrom100k            (chrom100k) int64 0 1 2 3 4 ... 27265 27266 27267 27268
    chrom100k_chrom      (chrom100k) object dask.array<shape=(27269,), chunksize=(27269,)>
    chrom100k_bin_start  (chrom100k) int64 dask.array<shape=(27269,), chunksize=(27269,)>
    chrom100k_bin_end    (chrom100k) int64 da

## Filter feature

### Remove chromosome

In [10]:
mcds = mcds.sel(chrom100k=~mcds.coords[f'{clustering_feature}_chrom'].isin(
    exclude_chromosome))

### Remove blacklist region

#### Get feature bed

In [11]:
feature_bed_df = pd.DataFrame([
    mcds.coords[f'{clustering_feature}_chrom'].to_pandas(),
    mcds.coords[f'{clustering_feature}_bin_start'].to_pandas(),
    mcds.coords[f'{clustering_feature}_bin_end'].to_pandas()
],
                              index=['chrom', 'start', 'end'],
                              columns=mcds.get_index(clustering_feature)).T
feature_bed = BedTool.from_dataframe(feature_bed_df)

#### Get black list bed

In [12]:
black_list_bed = BedTool(black_list_path)
black_feature = feature_bed.intersect(black_list_bed, f=0.2, wa=True)
black_feature_index = black_feature.to_dataframe().set_index(
    ['chrom', 'start', 'end']).index
black_feature_id = pd.Index(feature_bed_df.reset_index()\
                       .set_index(['chrom', 'start', 'end'])\
                       .loc[black_feature_index][clustering_feature])

In [13]:
mcds = mcds.sel(chrom100k=~mcds.get_index(clustering_feature).isin(black_feature_id))

### Remove by mean cov

In [14]:
# only calculate CHN

fig, axes = cutoff_vs_cell_remain(mcds[f'{clustering_feature}_da']\
                                  .sel(count_type='cov', mc_type='CHN')\
                                  .mean(axis=0)\
                                  .load())
fig.savefig(fig_dir / f'region_mean_CHN_cov.pdf')

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


In [15]:
before = mcds.coords[clustering_feature].size
mcds = mcds.filter_region_cov(dim=clustering_feature,
                              da=f'{clustering_feature}_da',
                              mc_type='CHN',
                              min_cov=min_feature_cov,
                              max_cov=max_feature_cov)
after = mcds.coords[clustering_feature].size
print(
    f'Filter {clustering_feature} by [{min_feature_cov}, {max_feature_cov}], '
    f'filter rate {(after / before):.2f}. {after} features remained.')

Filter chrom100k by [500, 3000], filter rate 0.99. 24050 features remained.


In [16]:
mcds

<xarray.MCDS>
Dimensions:              (cell: 853, chrom100k: 24050, count_type: 2, gene: 55487, mc_type: 2)
Coordinates:
  * mc_type              (mc_type) object 'CGN' 'CHN'
  * gene                 (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
  * count_type           (count_type) object 'mc' 'cov'
    strand_type          <U4 'both'
    gene_chrom           (gene) object dask.array<shape=(55487,), chunksize=(55487,)>
    gene_start           (gene) int64 dask.array<shape=(55487,), chunksize=(55487,)>
    gene_end             (gene) int64 dask.array<shape=(55487,), chunksize=(55487,)>
  * cell                 (cell) object '10E_M_1717' '10E_M_1724' ... '9J_M_794'
  * chrom100k            (chrom100k) int64 30 31 32 33 ... 26335 26336 26337
    chrom100k_chrom      (chrom100k) object dask.array<shape=(24050,), chunksize=(24050,)>
    chrom100k_bin_start  (chrom100k) int64 dask.array<shape=(24050,), chunksize=(24050,)>
    chrom100k_bin_end    (chrom100k) int64 dask.a

## Add mC rate

In [17]:
mcds.add_mc_rate(dim=clustering_feature, da=f'{clustering_feature}_da')

## Select HVF

### mCH HVF

In [18]:
ch_hvf_df = mcds.highly_variable_methylation_feature(
    var_dim=clustering_feature,
    mc_type='CHN',
    min_disp=None,
    max_disp=None,
    min_mean=min_ch_hvf_mean,
    max_mean=max_ch_hvf_mean,
    n_top_feature=ch_hvf_top,
    bin_min_features=5,
    mean_binsize=0.05,
    cov_binsize=100)

distributed.nanny - WARNING - Restarting worker


distributed.nanny - WARNING - Restarting worker


In [19]:
selection = ch_hvf_df['gene_subset']
print(f'Total Feature Number:     {selection.size}')
print(
    f'Highly Variable Feature:  {selection.sum()} ({(selection.sum() / selection.size * 100):.1f}%)'
)

fig, axes = plot_dispersion(ch_hvf_df,
                            hue='gene_subset',
                            zlab='dispersion_norm',
                            data_quantile=(0.01, 0.99),
                            save_animate_path=None,
                            fig_kws=None)
fig.savefig(fig_dir / f'ch_highly_variable_feature.{clustering_feature}.png',
            bbox_inches='tight')

Total Feature Number:     24050
Highly Variable Feature:  5999 (24.9%)


### mCG HVF

In [20]:
cg_hvf_df = mcds.highly_variable_methylation_feature(
    var_dim=clustering_feature,
    mc_type='CGN',
    min_disp=None,
    max_disp=None,
    min_mean=min_cg_hvf_mean,
    max_mean=max_cg_hvf_mean,
    n_top_feature=cg_hvf_top,
    bin_min_features=5,
    mean_binsize=0.01,
    cov_binsize=10)

In [21]:
selection = cg_hvf_df['gene_subset']
print(f'Total Feature Number:     {selection.size}')
print(
    f'Highly Variable Feature:  {selection.sum()} ({(selection.sum() / selection.size * 100):.1f}%)'
)

fig, axes = plot_dispersion(cg_hvf_df,
                            hue='gene_subset',
                            zlab='dispersion_norm',
                            data_quantile=(0.01, 0.99),
                            save_animate_path=None,
                            fig_kws=None)
fig.savefig(fig_dir / f'cg_highly_variable_feature.{clustering_feature}.png',
            bbox_inches='tight')

Total Feature Number:     24050
Highly Variable Feature:  6000 (24.9%)


## Get Anndata

### Load HVF

In [22]:
selected_ch_feature = ch_hvf_df[ch_hvf_df['gene_subset']].index
selected_cg_feature = cg_hvf_df[cg_hvf_df['gene_subset']].index

mcds = mcds.sel(chrom100k=(selected_ch_feature | selected_cg_feature))
mcds['chrom100k_da_rate'].load()

<xarray.DataArray 'chrom100k_da_rate' (cell: 853, chrom100k: 10647, mc_type: 2)>
array([[[1.096702, 1.210141],
        [1.133305, 0.923894],
        ...,
        [1.132073, 0.603346],
        [1.080031, 1.839382]],

       [[0.964761, 1.041648],
        [0.880692, 0.933607],
        ...,
        [0.999587, 1.153162],
        [0.993736, 0.855843]],

       ...,

       [[1.110708, 1.261325],
        [0.939115, 1.219443],
        ...,
        [0.920533, 0.843184],
        [0.491177, 0.813648]],

       [[0.835196, 0.725768],
        [1.082063, 0.882735],
        ...,
        [1.      , 1.      ],
        [1.      , 1.      ]]])
Coordinates:
  * mc_type              (mc_type) object 'CGN' 'CHN'
    strand_type          <U4 'both'
  * cell                 (cell) object '10E_M_1717' '10E_M_1724' ... '9J_M_794'
  * chrom100k            (chrom100k) int64 32 34 35 36 ... 26332 26333 26337
    chrom100k_chrom      (chrom100k) object 'chr1' 'chr1' ... 'chrX' 'chrX'
    chrom100k_bin_start  (chro

### CH adata

In [23]:
ch_adata = mcds.sel(chrom100k=selected_ch_feature).to_ann(
    f'{clustering_feature}_da_rate', var_dim=clustering_feature, mc_type='CHN')
ch_adata.write_h5ad(result_dir / 'mch_adata.norm_per_cell.hvf.h5ad')

Transforming to str index.


In [24]:
ch_adata

AnnData object with n_obs × n_vars = 853 × 5999 

### CG adata

In [25]:
cg_adata = mcds.sel(chrom100k=selected_cg_feature).to_ann(
    f'{clustering_feature}_da_rate', var_dim=clustering_feature, mc_type='CGN')
cg_adata.write_h5ad(result_dir / 'mcg_adata.norm_per_cell.hvf.h5ad')

Transforming to str index.


In [26]:
cg_adata

AnnData object with n_obs × n_vars = 853 × 6000 

## Prepare Gene

In [27]:
if clustering_feature != 'gene':
    mcds.add_gene_rate(in_memory=False,
                       output_prefix=str(result_dir / 'SelectedCell'),
                       cell_chunks=10000)